# Are Fandango Movie Ratings Inflated?

In October 2015, data journalist Walt Hickey discovered than the movie ratings on the [Fandango]() website were inflated by biased rounding. Instead of being rounded to the nearest half-star the ratings were rounded up. In some cases the ratings were even rounded up one whole star. Fandango later released a statement saying the rounding was an unintentional bug in the system that has now been fixed.

In this project we will investigate whether Fandango's ratings are still biased. We will use Hickey's original dataset and a new dataset collected by Dataquest for movies released in 2016 and 2017.

## Import Data & Install Dependencies

In [9]:
#Setup and reading in datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

pd.options.display.max_columns = 100  # Avoid having displayed truncated output
#pd.options.display.max_columns = 100  # Avoid having displayed truncated output

before = pd.read_csv('fandango_score_comparison.csv')
after = pd.read_csv('movie_ratings_16_17.csv')

## I. Explore the Data

In [10]:
display(before.sample(5))
display(after.sample(5))

,FILM,RottenTomatoes,RottenTomatoes_User,Metacritic,Metacritic_User,IMDB,Fandango_Stars,Fandango_Ratingvalue,RT_norm,RT_user_norm,Metacritic_norm,Metacritic_user_nom,IMDB_norm,RT_norm_round,RT_user_norm_round,Metacritic_norm_round,Metacritic_user_norm_round,IMDB_norm_round,Metacritic_user_vote_count,IMDB_user_vote_count,Fandango_votes,Fandango_Difference
16,Ted 2 (2015),46,58,48,6.5,6.6,4.5,4.1,2.30,2.90,2.40,3.25,3.30,2.5,3.0,2.5,3.5,3.5,197,49102,6437,0.4
125,The Wedding Ringer (2015),27,66,35,3.3,6.7,4.5,4.4,1.35,3.30,1.75,1.65,3.35,1.5,3.5,2.0,1.5,3.5,126,37292,6506,0.1
30,Red Army (2015),96,86,82,7.4,7.7,4.5,4.1,4.80,4.30,4.10,3.70,3.85,5.0,4.5,4.0,3.5,4.0,11,2275,54,0.4
135,The Lazarus Effect (2015),14,23,31,4.9,5.2,3.0,3.0,0.70,1.15,1.55,2.45,2.60,0.5,1.0,1.5,2.5,2.5,62,17691,1651,0.0
143,"Two Days, One Night (2014)",97,78,89,8.8,7.4,3.5,3.5,4.85,3.90,4.45,4.40,3.70,5.0,4.0,4.5,4.5,3.5,123,24345,118,0.0


,movie,year,metascore,imdb,tmeter,audience,fandango,n_metascore,n_imdb,n_tmeter,n_audience,nr_metascore,nr_imdb,nr_tmeter,nr_audience
19,Beauty and the Beast,2017,65,7.8,70,86,4.5,3.25,3.90,3.50,4.30,3.0,4.0,3.5,4.5
137,Ride Along 2,2016,32,5.9,14,49,4.0,1.60,2.95,0.70,2.45,1.5,3.0,0.5,2.5
80,Inferno,2016,42,6.2,19,37,3.5,2.10,3.10,0.95,1.85,2.0,3.0,1.0,2.0
123,Nocturnal Animals,2016,67,7.5,73,74,3.5,3.35,3.75,3.65,3.70,3.5,4.0,3.5,3.5
149,Star Trek Beyond,2016,68,7.1,84,81,4.5,3.40,3.55,4.20,4.05,3.5,3.5,4.0,4.0


In [12]:
# selecting only the columns that are relevant to Fandango ratings

fandango_before = before[['FILM', 'Fandango_Stars', 'Fandango_Ratingvalue', 'Fandango_votes', 'Fandango_Difference']].copy()
fandango_after = after[['movie', 'year', 'fandango']].copy()

display(fandango_before.sample(5))
display(fandango_after.sample(5))

,FILM,Fandango_Stars,Fandango_Ratingvalue,Fandango_votes,Fandango_Difference
73,Furious 7 (2015),5.0,4.8,33538,0.2
29,The End of the Tour (2015),4.5,4.1,121,0.4
101,The Gallows (2015),3.0,2.9,1896,0.1
67,Unfriended (2015),3.0,2.7,2507,0.3
4,Hot Tub Time Machine 2 (2015),3.5,3.0,1021,0.5


,movie,year,fandango
10,Anthropoid,2016,4.0
39,"Crouching Tiger, Hidden Dragon: Sword of Destiny",2016,4.0
143,Sing,2016,4.5
192,The Purge: Election Year,2016,4.0
9,Amateur Night,2016,3.5


The aim of the project is to check for any change in Fandango scores after the release of Hick's article. Which means our analysis population, ideally, would be all of the ratings on Fandango's website.

To perform the analysis we need to take two samples from the population: one before the analysis (Hick's data) and one after (our DataQuest dataset). We need to make sure these two samples are representative of the population we want to analyze.

The sampling conditions for Hick's data (according to the [README](https://github.com/fivethirtyeight/data/tree/master/fandango)):
- At least 30 fan ratings on Fandango at the time of sampling (24.08.2015)
- The movie must have had tickets for sale in 2015

The sampling was not done randomly. It might not be representative. It doesn't capture any time trends extending over the years; 2015 might have been an outlier.

The sampling conditions for the DataQuest data (according to the [README](https://github.com/mircealex/Movie_ratings_2016_17)):
- Only movies from 2016 or later
- Only the 214 most popular movies, determined by number of rating votes, were included.

This dataset sampling has the same problem with temporal trends and is unlikely to be representative of our population.

## II. Refining Project Goal

Based on the limitations discovered in the datasets samplings we will narrow down our initial goal.

Instead of attempting to determine if there was a change in Fandango's entire rating system after Hick's article, we will try to determine if there was a change in the rating of popular movies in 2015 and ratings for popular movies in 2016.

The new goal takes into account the limitations of our datasets, but is still fairly close to the original goal.

## Refining the Datasets

Based on our new goal of comparing popular movies on Fandango in 2015 with popular movies on Fandango in 2016 we will need to filter our datasets.

Hick's dataset used the criteria of 30 or more fan ratings to determine popularity. We will apply this same criteria to our DataQuest dataset. In other words, our definition of 'popular' will be movies with at least 30 fan ratings on Fandango.

### Filtering `fandango_before` Dataset

In [26]:
fandango_before['Year'] = fandango_before['FILM'].str[-5:-1]
fandango_before.head(3)

,FILM,Fandango_Stars,Fandango_Ratingvalue,Fandango_votes,Fandango_Difference,Year
0,Avengers: Age of Ultron (2015),5.0,4.5,14846,0.5,2015
1,Cinderella (2015),5.0,4.5,12640,0.5,2015
2,Ant-Man (2015),5.0,4.5,12055,0.5,2015


In [28]:
fandango_before['Year'].value_counts()

2015    129
2014     17
Name: Year, dtype: int64

In [36]:
fandango_2015 = fandango_before[fandango_before['Year'] == '2015'].copy()
fandango_2015['Year'].value_counts()

2015    129
Name: Year, dtype: int64

### Filtering `fandango_after` Dataset

In [42]:
display(fandango_after.sample(3))
display((fandango_after['year'].value_counts()))

,movie,year,fandango
156,Teenage Mutant Ninja Turtles: Out of the Shadows,2016,4.0
78,Incarnate,2016,3.0
57,Florence Foster Jenkins,2016,4.0


2016    191
2017     23
Name: year, dtype: int64

In [41]:
fandango_2016 = fandango_after[fandango_after['year'] == 2016]
fandango_2016['year'].value_counts()

2016    191
Name: year, dtype: int64

## Checking the Datasets

Using Hick's benchmark of a minimum of 30 fan ratings let's check if the datasets contain only information that matches this.

In [15]:
fandango_before['Fandango_votes'].min()

35

The minimum nr. of fan ratings in Hick's data is 35. All films in the dataset can therefore be classified as popular.

Next, we check the dataset collected by DataQuest. However, this dataset does not contain information about the nr. of votes.

Instead we will sample 10 films randomly from the dataset and check the films manually. At least 8 out 10 samples should match our criteria.

We can't check against Fandango anymore since they replaced their rating system with the Rotten Tomatoes one in 2019. We will instead check the films against the Rotten Tomatoes ratings.



In [43]:
fandango_2016.sample(10, random_state=1)

,movie,year,fandango
36,Collide,2016,3.5
75,Ice Age: Collision Course,2016,4.0
125,Now You See Me 2,2016,4.0
171,The Disappointments Room,2016,2.5
92,Kubo and the Two Strings,2016,4.5
134,Pride and Prejudice and Zombies,2016,4.0
40,Deadpool,2016,4.5
115,Money Monster,2016,4.0
204,War Dogs,2016,4.0
111,Miracles from Heaven,2016,4.5
